In [2]:
import pandas as pd
import requests

In [4]:
url = 'https://api.usa.gov/crime/fbi/cde/hate-crime/national?from=10-2022&to=10-2023&type=rates&API_KEY=6pQ1XOFvDCRG0DpNsdgIhcyiCmfUafzMzPETHn3K'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code}")

{'bias_section': {'victim_type': {'Other': 342, 'Unknown': 48, 'Business': 1125, 'Government': 766, 'Individual': 13272, 'Society/Public': 273, 'Financial Institution': 4, 'Religious Organization': 300, 'Law Enforcement Officer': 248}, 'offense_type': {'Rape': 17, 'Arson': 60, 'Incest': 0, 'Sodomy': 5, 'Bribery': 1, 'Robbery': 164, 'Wire Fraud': 5, 'Shoplifting': 88, 'Embezzlement': 4, 'Intimidation': 4842, 'Prostitution': 1, 'Impersonation': 7, 'Not Specified': 9, 'Welfare Fraud': 2, 'Identity Theft': 20, 'Pocket-picking': 4, 'Simple Assault': 3259, 'Statutory Rape': 1, 'Purse-snatching': 4, 'Betting/Wagering': 0, 'Sports Tampering': 0, 'All Other Larceny': 445, 'Aggravated Assault': 1652, 'Extortion/Blackmail': 21, 'Motor Vehicle Theft': 50, 'Theft From Building': 70, 'Kidnapping/Abduction': 17, 'Weapon Law Violations': 97, 'Counterfeiting/Forgery': 8, 'Negligent Manslaughter': 0, 'Criminal Sexual Contact': 34, 'Purchasing Prostitution': 0, 'Drug/Narcotic Violations': 112, 'Stolen Pr